In [287]:
import pickle

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import Dataset, DataLoader
from torch.nn import Embedding, RNN
from sklearn.model_selection import train_test_split
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

#Below imports are via https://www.datacamp.com/tutorial/parameter-optimization-machine-learning-models
import random

# Set any global values

In [288]:
torch.manual_seed(1)

In [289]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


# Load the training data

In [290]:
with open('training_data.pickle','rb') as f:
    training_data = pickle.load(f)

In [291]:
class_to_ix = {"Found":0, "Unfound":1}
all_letters = "()ab~&|>"

vocab = ['<pad>'] + sorted(set([char for seq in all_letters for char in seq]))


n_classes = len(class_to_ix)
n_letters = len(all_letters)

In [292]:
X = []
Y = []
embed_dim = len(vocab)
embed = Embedding(len(vocab), embed_dim) # embedding_dim = len(vocab)
for element in training_data:
    input = [vocab.index(token) for token in element[0]]
    input_tensor = torch.tensor(input, dtype=torch.int)
    classification = class_to_ix[element[1]]
    X.append(input_tensor)
    output_tensor = torch.tensor(classification, dtype=torch.int)
    Y.append(output_tensor)

In [293]:
class TTPStyleDataset(Dataset):
    def __init__(self, X,Y):
        self.X = X
        self.Y = Y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        sample = [self.X[idx], self.Y[idx]]
        return sample

In [294]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

batch_size = 16
input_dataset = TTPStyleDataset(X_train,y_train)

test_dataset = TTPStyleDataset(X_test,y_test)

def my_collate_fn(data):
    (xx,yy) = zip(*data)
    x_lens = [len(x) for x in xx]
    y_lens = [1 for y in yy]

    xx_pad = pad_sequence(xx, batch_first=False, padding_value=0)
    yy_pad = torch.tensor(yy)

    return xx_pad, yy_pad, x_lens, y_lens
    

#dataloader = DataLoader(input_dataset, shuffle=True, batch_size=batch_size, num_workers=0, collate_fn=my_collate_fn)
#test_dataloader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size, num_workers=0, collate_fn=my_collate_fn)

# Neural Network

In [295]:
class SoftmaxRNN(nn.RNN):
    def __init__(self, input_size, hidden_size, num_layers, nonlinearity, bias, batch_first, dropout, bidirectional, output_size):
        super(SoftmaxRNN, self).__init__(input_size, hidden_size, num_layers, nonlinearity, bias, batch_first, dropout, bidirectional)
        self.fc = nn.Linear(hidden_size, output_size)
        
    
    def forward(self, x, h_0=None):
        output, h = super(SoftmaxRNN, self).forward(x, h_0)
        output, seq_len = pad_packed_sequence(output, batch_first=False)
        output = self.fc(h[-1])
        return output, h

In [296]:
#input_size - The number of expected features in the input x
rnn = SoftmaxRNN(input_size=embed_dim, hidden_size=128, num_layers=1, nonlinearity='relu', bias=True, batch_first=False, dropout=0.0, bidirectional=False, output_size=2)
rnn.to(device)

SoftmaxRNN(
  9, 128
  (fc): Linear(in_features=128, out_features=2, bias=True)
)

# GRU

Referencing "Dive into Deep Learning" by Aston Zhang, et al. pages 376-382.
Referencing https://medium.com/@anishnama20/understanding-gated-recurrent-unit-gru-in-deep-learning-2e54923f3e2

__Below are some notes.__

GRU contains the following.
* Two gates in which are the **Reset** and **Update** gates. 
* Candidate Hidden State - Combines information from the input and the previous hidden state that is used to update the hidden state for the next time step. The Long & Short Term memory are now in this candidate hidden state(?)

Reset Gate - Determines how much of the previous hidden state to forget. 
* Takes hidden state h_t-1 and current word x_t apply
Update Gate - Determines how much of the candidate hidden state to incorprate into the new hidden state.

A bit of recap on Gaussian Distribution: https://stackoverflow.com/questions/12616406/anyone-can-tell-me-why-we-always-use-the-gaussian-distribution-in-machine-learni


In [297]:
#https://d2l.ai/chapter_recurrent-modern/gru.html#implementation-from-scratch
class GRU(nn.GRU):
    #Input size - input_size - Defines the number of features that define each element (time-stamp) of the input sequence.
        #This passes the features into the hidden layers that will perform computations. 
    #hidden_size - Defines the size (amount of features) of the hidden state. Therefore, if hidden_size is set as 4, then the hidden state at each time step is a vector of length 4
    def __init__(self, input_size, hidden_size, num_layers, bias, batch_first, dropout, bidirectional, output_size): #Why is it if I have num_layers = 2, we get an error? 
        super(GRU, self).__init__(input_size, hidden_size, num_layers, bias, batch_first, dropout, bidirectional) #Initializing parent class of our GRU class (in this case it is torch.nn.GRU)
        self.gru = nn.GRU(input_size, hidden_size)
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, input, h_0 = None):
        output, h = super(GRU, self).forward(input, h_0) #Returns packed sequence only
        output, seq_len = pad_packed_sequence(output, batch_first=False)
        output = self.fc(h[-1])
        return output, h

In [298]:
input_size = embed_dim
hidden_size = 128
num_layers = 3
bias = True
batch_first = False
dropout = 0.0
bidirectional = False
output_size = 2

rnn = GRU(input_size, hidden_size, num_layers, bias, batch_first, dropout, bidirectional, output_size)
rnn.to(device)

GRU(
  9, 128, num_layers=3
  (gru): GRU(9, 128)
  (fc): Linear(in_features=128, out_features=2, bias=True)
)

# Output best to worst trials

In [299]:
list_of_params = []
def add_to_list_of_params(entry):
    list_of_params.append(entry)
    list_of_params.sort(key=lambda x: x["accuracy"], reverse=True)
    
def print_all_trials():
    print("=========================\nTrials from Best to Worst\n=========================\n")
    for i, entry in enumerate(list_of_params, 1):
        print(f"{i}: "
              f"Trial #{entry['trial']}, "
              f"Accuracy = {entry['accuracy']}, "
              f"Avg Test Loss = {entry['avg_test_loss']}, "
              f"Hidden Size = {entry['hidden_size']}, "
              f"Num Layers = {entry['num_layers']}, "
              f"Bias = {entry['bias']}, "
              f"Batch First = {entry['batch_first']}, "
              f"Dropout = {entry['dropout']}, "
              f"Bidirectional = {entry['bidirectional']}, "
              f"Num Epochs = {entry['num_epochs']}, "
              f"Learning Rate = {entry['learning_rate']}, "
              f"Batch Size = {entry['batch_size']}")


# Training with Trials

In [300]:
num_trials = 15
for trial in range(num_trials):
    best_accuracy = 0
    best_params = None

    input_size = embed_dim
    hidden_size = random.choice([32, 64, 128, 256])
    num_layers = random.choice([2,3,4,5])
    bias = random.choice([True, False])
    batch_first = random.choice([True, False])
    dropout = 0.0    
    bidirectional = random.choice([True, False])
    output_size = 2
    num_epochs = random.choice([10, 11, 12, 13, 14, 15])
    #num_epochs = random.choice([1])
    learning_rate = random.choice([0.01, 0.001]) # If you set this too high, it might explode. If too low, it might not learn

    #Loading in data!
    dataloader = DataLoader(input_dataset, shuffle=True, batch_size=batch_size, num_workers=0, collate_fn=my_collate_fn)
    test_dataloader = DataLoader(test_dataset, shuffle=True, batch_size=batch_size, num_workers=0, collate_fn=my_collate_fn)

    rnn = GRU(input_size, hidden_size, num_layers, bias, batch_first, dropout, bidirectional, output_size)
    rnn.to(device)

    optimizer = torch.optim.SGD(rnn.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()

    print("==============================================================================================",
        "\nTrial", str(trial + 1), "\n"
        #"\noptimizer: ", optimizer, "\ncriterion: ", criterion,
        "\nhidden_size: ", hidden_size, "\nnum_layers: ", num_layers, 
        "\nbias = ", bias, "\nbatch_first = ", batch_first, 
        "\ndropout = ", dropout, "\nbidirectional = ", bidirectional, 
        "\nnum_epochs = ", num_epochs, "\nlearning_rate = ", learning_rate, 
        "\nbatch_size = ", batch_size,
        "\n=========================")

    for epoch in range(num_epochs):
        print("epoch: "+str(epoch))
        
        size = len(dataloader.dataset)
        rnn.train()
        for batch,(x_padded, y_padded, x_lens, y_lens) in enumerate(dataloader):

            
            rnn.zero_grad()
            x_embed = embed(x_padded)
            x_packed = pack_padded_sequence(x_embed, x_lens, batch_first=False, enforce_sorted=False)
            x_packed = x_packed.to(device)
            y_padded = y_padded.to(device)
            output, hidden = rnn(x_packed)
            output = torch.reshape(output, (batch_size,2))
            
            y_padded = torch.reshape(y_padded, (batch_size,))
            y_padded = y_padded.long()
            loss = criterion(output, y_padded)

            loss.backward()
            optimizer.step()
            

            if batch % 5000 == 0:
                x_s = np.shape(x_padded)[1]
                loss, current = loss.item(), (batch + 1) * x_s
                print(f"train loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")



        # Set the model to evaluation mode - important for batch normalization and dropout layers
        # Unnecessary in this situation but added for best practices
        rnn.eval()
        size = len(test_dataloader.dataset)
        num_batches = len(test_dataloader)
        test_loss, correct = 0, 0

    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
        # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
        with torch.no_grad():
            for batch,(x_padded, y_padded, x_lens, y_lens) in enumerate(test_dataloader):
                rnn.zero_grad()
                x_embed = embed(x_padded)
                x_packed = pack_padded_sequence(x_embed, x_lens, batch_first=False, enforce_sorted=False)
                x_packed = x_packed.to(device)
                y_padded = y_padded.to(device)
                output, hidden = rnn(x_packed)
                output = torch.reshape(output, (batch_size,2))
                
                y_padded = torch.reshape(y_padded, (batch_size,))
                y_padded = y_padded.long()
                correct += (output.argmax(1) == y_padded).type(torch.float).sum().item()
                loss += criterion(output, y_padded)
            
            #loss, current = loss, (batch + 1) * len(x_padded)
            test_loss = loss/num_batches
            correct /= size
            accuracy = 100*correct
            print(f"Test Error: \n Accuracy: {accuracy:>0.1f}%, Avg loss: {test_loss:>8f} \n") 
    hyperparams = {
        'trial': (trial+1),
        'accuracy': accuracy,
        'avg_test_loss': test_loss,
        'hidden_size': hidden_size,
        'num_layers': num_layers,
        'bias': bias,
        'batch_first': batch_first,
        'dropout': dropout, 
        'bidirectional': bidirectional,
        'num_epochs': num_epochs,
        'learning_rate': learning_rate, 
        'batch_size': batch_size
    }
    add_to_list_of_params(hyperparams)
    print_all_trials()

    '''
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_params = {
            'hidden_size': hidden_size,
            'num_layers': num_layers,
            'bias': bias,
            'batch_first': batch_first,
            'dropout': dropout, 
            'bidirectional': bidirectional,
            'num_epochs': num_epochs,
            'learning_rate': learning_rate, 
            'batch_size': batch_size
        }
    '''

#print(f"Best Accuracy: {best_accuracy:.2f}%")
#print(f"Best Parameters: {best_params}")


Trial 1 

hidden_size:  64 
num_layers:  2 
bias =  False 
batch_first =  True 
dropout =  0.0 
bidirectional =  True 
num_epochs =  13 
learning_rate =  0.01 
batch_size =  16 
epoch: 0
train loss: 0.691025  [   16/141120]
train loss: 0.527832  [80016/141120]
Test Error: 
 Accuracy: 70.6%, Avg loss: 0.595550 

epoch: 1
train loss: 0.633524  [   16/141120]
train loss: 0.627528  [80016/141120]
Test Error: 
 Accuracy: 70.7%, Avg loss: 0.594137 

epoch: 2
train loss: 0.701585  [   16/141120]
train loss: 0.594272  [80016/141120]
Test Error: 
 Accuracy: 70.8%, Avg loss: 0.593338 

epoch: 3
train loss: 0.853814  [   16/141120]
train loss: 0.484992  [80016/141120]
Test Error: 
 Accuracy: 71.0%, Avg loss: 0.591596 

epoch: 4
train loss: 0.477034  [   16/141120]
train loss: 0.592743  [80016/141120]
Test Error: 
 Accuracy: 71.0%, Avg loss: 0.590618 

epoch: 5
train loss: 0.833911  [   16/141120]
train loss: 0.602738  [80016/141120]
Test Error: 
 Accuracy: 70.4%, Avg loss: 0.582781 

epoch: 6
tra

# Training (Original)

In [301]:
'''
num_epochs = 13
learning_rate = 0.01 # If you set this too high, it might explode. If too low, it might not learn
optimizer = torch.optim.SGD(rnn.parameters(), lr=learning_rate)

criterion = nn.CrossEntropyLoss()

print("===============================================",
      #"\noptimizer: ", optimizer, "\ncriterion: ", criterion,
      "\nhidden_size: ", hidden_size, "\nnum_layers: ", num_layers, 
      "\nbias = ", bias, "\nbatch_first = ", batch_first, 
      "\ndropout = ", dropout, "\nbidirectional = ", bidirectional, 
      "\nnum_epochs = ", num_epochs, "\nlearning_rate = ", learning_rate, 
      "\nbatch_size = ", batch_size,
      "\n===============================================")

for epoch in range(num_epochs):
    print("epoch: "+str(epoch))
    size = len(dataloader.dataset)
    rnn.train()
    for batch,(x_padded, y_padded, x_lens, y_lens) in enumerate(dataloader):

        
        rnn.zero_grad()
        x_embed = embed(x_padded)
        x_packed = pack_padded_sequence(x_embed, x_lens, batch_first=False, enforce_sorted=False)
        x_packed = x_packed.to(device)
        y_padded = y_padded.to(device)
        output, hidden = rnn(x_packed)
        output = torch.reshape(output, (batch_size,2))
        
        y_padded = torch.reshape(y_padded, (batch_size,))
        y_padded = y_padded.long()
        loss = criterion(output, y_padded)

        loss.backward()
        optimizer.step()
        

        if batch % 5000 == 0:
            x_s = np.shape(x_padded)[1]
            loss, current = loss.item(), (batch + 1) * x_s
            print(f"train loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")



    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    rnn.eval()
    size = len(test_dataloader.dataset)
    num_batches = len(test_dataloader)
    test_loss, correct = 0, 0

 # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to reduce unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
        for batch,(x_padded, y_padded, x_lens, y_lens) in enumerate(test_dataloader):
            rnn.zero_grad()
            x_embed = embed(x_padded)
            x_packed = pack_padded_sequence(x_embed, x_lens, batch_first=False, enforce_sorted=False)
            x_packed = x_packed.to(device)
            y_padded = y_padded.to(device)
            output, hidden = rnn(x_packed)
            output = torch.reshape(output, (batch_size,2))
            
            y_padded = torch.reshape(y_padded, (batch_size,))
            y_padded = y_padded.long()
            correct += (output.argmax(1) == y_padded).type(torch.float).sum().item()
            loss += criterion(output, y_padded)
        
        #loss, current = loss, (batch + 1) * len(x_padded)
        test_loss = loss/num_batches
        correct /= size
        print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n") 

'''

'\nnum_epochs = 13\nlearning_rate = 0.01 # If you set this too high, it might explode. If too low, it might not learn\noptimizer = torch.optim.SGD(rnn.parameters(), lr=learning_rate)\n\ncriterion = nn.CrossEntropyLoss()\n\nprint("===============================================",\n      #"\noptimizer: ", optimizer, "\ncriterion: ", criterion,\n      "\nhidden_size: ", hidden_size, "\nnum_layers: ", num_layers, \n      "\nbias = ", bias, "\nbatch_first = ", batch_first, \n      "\ndropout = ", dropout, "\nbidirectional = ", bidirectional, \n      "\nnum_epochs = ", num_epochs, "\nlearning_rate = ", learning_rate, \n      "\nbatch_size = ", batch_size,\n      "\n===============================================")\n\nfor epoch in range(num_epochs):\n    print("epoch: "+str(epoch))\n    size = len(dataloader.dataset)\n    rnn.train()\n    for batch,(x_padded, y_padded, x_lens, y_lens) in enumerate(dataloader):\n\n        \n        rnn.zero_grad()\n        x_embed = embed(x_padded)\n        x_